**Notebook to query multiple sources within the ALMA archival data** \
More information on querying the ALMA archive programatically: \
https://almascience.eso.org/alma-data/archive/archive-notebooks \
Documentation for *astroquery*: \
https://astroquery.readthedocs.io/

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astroquery.simbad import Simbad
from astroquery.alma import Alma
from astropy import coordinates
from astropy.io import fits
import astropy.units as u
import aplpy
from astropy.table import Table
from astropy.io import ascii

In [2]:
# Read in table. Note: pandas read_csv and astropy.table are other options
catalog = ascii.read("galaxy-catalog.csv", guess='ecsv', delimiter=',')  
print(catalog['Name'])

  Name  
--------
NGC_1068
Hen_2-10
Antennae


In [3]:
# Define parameters for data query
band_list = [6]
search_radius = 30*u.arcsec
for index, target in enumerate(catalog['Name']):
    alma = Alma()
    print('#########################################################################')
    print('Querying {}'.format(target))
    print('#########################################################################')
    c = coordinates.SkyCoord(ra=catalog[index]['RA (hms)'], dec=catalog[index]['Dec (dms)'], unit=(u.hourangle, u.deg),frame='icrs')
    #print(c)
    # Make data query: 
    alma_data = alma.query_region(c, search_radius, band_list=band_list).to_pandas()
    uids = np.unique(alma_data['member_ous_uid'])
    print(uids)
    link_list = alma.get_data_info(uids, expand_tarfiles=True)
    print('Total number of links available for download {}'.format(len(link_list)))
    print(link_list)
    print(alma_data)
    alma_data.to_csv(f'{target}_query_region.csv')
    print('\n')

#########################################################################
Querying NGC_1068
#########################################################################


ConnectTimeout: HTTPSConnectionPool(host='almascience.nrao.edu', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f9c29e94710>, 'Connection to almascience.nrao.edu timed out. (connect timeout=60)'))